In [1]:
%matplotlib widget
import datetime
import tensorflow as tf
import numpy as np
import pandas as pd
import xarray as xr
import time
import gym
import matplotlib.pyplot as plt
import cmocean
#from simple_agent import SimpleAtariAgent
from tqdm import tqdm
import random
import dqn_agent

Using TensorFlow backend.


In [2]:
env = gym.make('gym_biomapping:perfect_info_atari-v0', static=True, output='3D-matrix')
agent = dqn_agent.DQNAgent(env, input="3D-matrix")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Reset the position to (0,0,0)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [3]:
obs = env.reset()
next_action = agent.deliberate(obs)
agent.render(obs, next_action)
env.render()

Reset the position to (0,0,0)


In [ ]:
done = False
while not done:
    obs, reward, done, info = env.step(next_action)
    #next_pos = agent.deliberate(obs)
    agent.render(obs, next_action)
    env.render()

In [4]:
# For more reproduceable results, but consider changing
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)

In [5]:
# This is where the magic happens
EPISODES = 100
AGGREGATE_STATS_EVERY = 1 # Episode
ep_rewards = []
SHOW_PREVIEW = True
MIN_REWARD = -10_000 # Rewards below this are not saved
MODEL_NAME = "LAUV_ROALD"

# Epsilon parameters determine the extent to which the agent will explore/exploit
    # Using decaying epsilon: The more the agent learns, the less it will explore
    # epsilon := Change of doing explore action, i.e. randomly select action
    # EPSILON_DECAY := The rate at which exploring decays
    # MIN_EPSILON := To always keep some level of exploration, exploration will not decay beyond this threshold

epsilon = 1 # Not constant, as it will decay
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001


for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
    
    # Update tensorboard step every episode
    agent.tensorboard.step = episode
    
    # Reset episode, rewards, environment, done-flag
    episode_reward = 0
    step = 1
    current_state = env.reset()
    agent.reset()
    done = False
    
    # The training loop
    while not done:
        # Decide explore vs exploit
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.nvec[0])

        # Take selected aciton
        new_state, reward, done, _info = env.step([action])
        next_pos = agent.deliberate(new_state)
        
        episode_reward+=reward
    
        if SHOW_PREVIEW:
            agent.render(new_state, next_action)
            env.render()

        # Update replay-memory (a set length collection of experiences the agent remembers, from which training examples are selected)
        # Works as a deque: one in, oldest out
        #print("Updating replay memory...")
        agent.update_replay_memory((current_state, action, reward, new_state, done))

        # Trains the agent with the replay-memory is sufficiently large
        agent.train(done, step)
        current_state = new_state
        step += 1
        
    print("Episode reward: ", episode_reward)
    
    # Decay epsilon
    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)
        
    # Logging and storing
    ep_rewards.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if min_reward >= MIN_REWARD:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')
    
        

  0%|                      | 0/100 [00:00<?, ?episodes/s]

Reset the position to (0,0,0)
Episode reached max number of steps
Episode reward:  109.02322769165039



  1%|1             | 1/100 [00:11<19:14, 11.66s/episodes]

Reset the position to (0,0,0)
Episode reached max number of steps


  2%|2             | 2/100 [00:23<18:55, 11.59s/episodes]

...

Episode reward:  263.69483375549316
Reset the position to (0,0,0)
Episode reached max number of steps


 99%|##########8| 99/100 [1:13:22<00:55, 55.12s/episodes]

Episode reward:  216.68279647827148
Reset the position to (0,0,0)
Episode reached max number of steps


100%|##########| 100/100 [1:14:16<00:00, 44.57s/episodes]

Episode reward:  325.525297164917
